In [1]:
import tensorflow as tf
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from keras.preprocessing.image import ImageDataGenerator

C:\Users\agnes\anaconda3\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
C:\Users\agnes\anaconda3\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.10.1 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure yo

In [2]:
### how do i even open all the images ,what do i use 
## im trying to read some papers on MobileNet, ResNet, should i look at. not implementing the algorithm, just using 
# look at tf, keras. traingenerator 
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   fill_mode="nearest")

In [3]:
trainImageData = train_datagen.flow_from_directory("C:\\Users\\agnes\\Documents\\apziva_ai_residency\\project4\\data\\images\\training",
                                                     batch_size=32, #how much images to give per iteration in an epoch
                                                     class_mode="binary", #Incase of multi-class classification, "categorical"
                                                     target_size=(64,64) #Ensures all images are of same size (resizing)
                                                     ) 
testImageData = train_datagen.flow_from_directory("C:\\Users\\agnes\\Documents\\apziva_ai_residency\\project4\\data\\images\\testing",
                                                     batch_size=32, #how much images to give per iteration in an epoch
                                                     class_mode="binary", #Incase of multi-class classification, "categorical"
                                                     target_size=(64,64) #Ensures all images are of same size (resizing)
                                                     ) 

Found 2392 images belonging to 2 classes.
Found 597 images belonging to 2 classes.


In [4]:
## load ResNet from keras to perform transfer learning
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Flatten
from tensorflow.keras.applications import ResNet152
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
# Importing the ResNet model but discarding the last 1000 neuron layer for adjustment to our dataset
base_model = tf.keras.applications.ResNet152(weights = 'imagenet', include_top = False, input_shape=(64, 64, 3))

234698864/234698864 [==============================] - 8s 0us/step


In [6]:
## freeze the base_model so we keep the pre-trained structure
base_model.trainable = False

In [7]:
## adjust model to our own dataset
model = tf.keras.models.Sequential([
    base_model,
    Flatten(),
    Dense(1024, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet152 (Functional)      (None, 2, 2, 2048)        58370944  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 1024)              8389632   
                                                                 
 dense_1 (Dense)             (None, 1)                 1025      
                                                                 
Total params: 66,761,601
Trainable params: 8,390,657
Non-trainable params: 58,370,944
_________________________________________________________________


In [9]:
## compile 
model.compile(optimizer="adam",
              loss="binary_crossentropy", #For multi-class classification: categorical_crossentropy | sparse_categorical_crossentropy
              metrics=[tfa.metrics.F1Score(num_classes=1, threshold=0.5)])

In [10]:
len(trainImageData.filenames) // trainImageData.batch_size


74

In [12]:
# Fit the model
model.fit(trainImageData,
          validation_data=testImageData,
          epochs=10,
          steps_per_epoch = len(trainImageData.filenames) // trainImageData.batch_size,
          validation_steps= len(testImageData.filenames) // testImageData.batch_size)

Epoch 1/10
74/74 [==============================] - 78s 989ms/step - loss: 0.8179 - f1_score: 0.5660 - val_loss: 0.7047 - val_f1_score: 0.6570
Epoch 2/10
74/74 [==============================] - 70s 946ms/step - loss: 0.6998 - f1_score: 0.5705 - val_loss: 0.6964 - val_f1_score: 0.3664
Epoch 3/10
74/74 [==============================] - 70s 947ms/step - loss: 0.6745 - f1_score: 0.5887 - val_loss: 0.6822 - val_f1_score: 0.5520
Epoch 4/10
74/74 [==============================] - 70s 951ms/step - loss: 0.6711 - f1_score: 0.6210 - val_loss: 0.6583 - val_f1_score: 0.4957
Epoch 5/10
74/74 [==============================] - 70s 947ms/step - loss: 0.6655 - f1_score: 0.6108 - val_loss: 0.6721 - val_f1_score: 0.4802
Epoch 6/10
74/74 [==============================] - 73s 985ms/step - loss: 0.6605 - f1_score: 0.6275 - val_loss: 0.6554 - val_f1_score: 0.5943
Epoch 7/10
74/74 [==============================] - 70s 949ms/step - loss: 0.6463 - f1_score: 0.6361 - val_loss: 0.6399 - val_f1_score: 0.6897

In [15]:
model.save('ResNet.keras')


In [ ]:
## Model is massive --> 327 MB, not for phone